## Initial Setup

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1663564264150_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Run this everytime you create a new spark instance. 

spark.sparkContext.install_pypi_package("plotly==5.5.0")
spark.sparkContext.install_pypi_package("pandas==0.25.1")
spark.sparkContext.install_pypi_package("numpy==1.14.5")
spark.sparkContext.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1663564450526-0


You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [113]:
# Write all import commands here 
# importing window from pyspark.sql.window
from pyspark.sql.window import Window
from pyspark.sql.functions import col,avg,sum,min,max,row_number, count, when, isnan
from pyspark.sql import functions as F
import sys
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = sqlContext.createDataFrame( [(1,2,"a"),(3,2,"a"),(1,3,"b"),(2,2,"a"),(2,3,"b")], 
                                 ["time", "value", "class"] )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql import Window
from pyspark.sql import functions as F

windowval = (Window.partitionBy('class').orderBy('time')
             .rangeBetween(Window.unboundedPreceding, 0))
df_w_cumsum = df.withColumn('cum_sum', F.sum('value').over(windowval))
df_w_cumsum.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+-------+
|time|value|class|cum_sum|
+----+-----+-----+-------+
|   1|    2|    a|      2|
|   2|    2|    a|      4|
|   3|    2|    a|      6|
|   1|    3|    b|      3|
|   2|    3|    b|      6|
+----+-----+-----+-------+

In [10]:
df_w_cumsum.filter('cum_sum > 3').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----+-------+
|time|value|class|cum_sum|
+----+-----+-----+-------+
|   2|    2|    a|      4|
|   3|    2|    a|      6|
|   2|    3|    b|      6|
+----+-----+-----+-------+

In [6]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array([1, 8])
ypoints = np.array([3, 10])
print('xpoints are - ', xpoints)

plt.plot(xpoints, ypoints)
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xpoints are -  [1 8]

## Defining Custom Functions 

In [11]:
def get_quantiles(df, col_name, quantiles_list=[0.01, 0.25, 0.5, 0.75, 0.99]):
    """
    Takes a numerical column and returns column values at requested quantiles

    Inputs 
    Argument 1: Dataframe
    Argument 2: Name of the column
    Argument 3: A list of quantiles you want to find. Default value [0.01, 0.25, 0.5, 0.75, 0.99]

    Output 
    Returns a dictionary with quantiles as keys and column quantile values as values 
    """
    # Get min, max and quantile values for given column
    zero = df.approxQuantile(col_name,[0.0] , 0.2)[0]
    quant_vals = df.approxQuantile(col_name, quantiles_list, 0.2)
    one = df.approxQuantile(col_name,[1.0] , 0.2)[0]

    # Store min, quantiles and max in output dict, sequentially
    quantiles_dict = {0.0:int(zero)}
    for i in range(0,len(quantiles_list)):
        quantiles_dict[quantiles_list[i]]=quant_vals[i]
    quantiles_dict[1.0]=int(one)

    return(quantiles_dict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
def add_OHE_columns (df, n_name_list):
    """
    Given a list of tags, creates one hot encoded columns for each tag. 
  
    Input
    Argument 1: Dataframe in which the function will add the new columns
    Argument 2: list of tags
  
    Output
    Prints the names of columns that have been added 
    Returns the modified dataframe 
    """
    # write code to create encoded columns 
    for name in n_name_list:
        opt_col = name+'_numeric'
        indexer = StringIndexer(inputCol=name, outputCol=opt_col)
        df = indexer.fit(df).transform(df)
        oht_opt_col = name+'_onehot'
        encoder = OneHotEncoder(inputCols=['opt_col'], outputCols=[oht_opt_col])
        df = encoder.fit(df).transform(df)
        df = df.select('*', vector_to_array('oht_opt_col').alias('col_onehot'))        
        print ("added column: has_tag_"+name)

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
df = add_OHE_columns(interaction_level_df, top_frequent_tags_list)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1317.fit.
: org.apache.spark.SparkException: Input column easy does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:245)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:245)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:242)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apach

In [116]:
#sample reference - https://www.skytowner.com/explore/one_hot_encoding_in_pyspark

rows = [['Alex','B'], ['Bob','A'], ['Cathy','B'], ['Dave','C'], ['Eric','D']]
df = spark.createDataFrame(rows, ['name','class'])
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
| name|class|
+-----+-----+
| Alex|    B|
|  Bob|    A|
|Cathy|    B|
| Dave|    C|
| Eric|    D|
+-----+-----+

In [117]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='class', outputCol='class_numeric')
df_indexed = indexer.fit(df).transform(df)
df_indexed.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+-------------+
| name|class|class_numeric|
+-----+-----+-------------+
| Alex|    B|          0.0|
|  Bob|    A|          1.0|
|Cathy|    B|          0.0|
| Dave|    C|          2.0|
| Eric|    D|          3.0|
+-----+-----+-------------+

In [120]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=['class_numeric'], outputCols=["oht_opt_col"])
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+-------------+-------------+
| name|class|class_numeric|  oht_opt_col|
+-----+-----+-------------+-------------+
| Alex|    B|          0.0|(3,[0],[1.0])|
|  Bob|    A|          1.0|(3,[1],[1.0])|
|Cathy|    B|          0.0|(3,[0],[1.0])|
| Dave|    C|          2.0|(3,[2],[1.0])|
| Eric|    D|          3.0|    (3,[],[])|
+-----+-----+-------------+-------------+

In [121]:
from pyspark.ml.functions import vector_to_array
df_col_onehot = df_onehot.select('*', vector_to_array('oht_opt_col').alias('col_onehot'))
df_col_onehot.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+-------------+-------------+---------------+
| name|class|class_numeric|  oht_opt_col|     col_onehot|
+-----+-----+-------------+-------------+---------------+
| Alex|    B|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
|  Bob|    A|          1.0|(3,[1],[1.0])|[0.0, 1.0, 0.0]|
|Cathy|    B|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
| Dave|    C|          2.0|(3,[2],[1.0])|[0.0, 0.0, 1.0]|
| Eric|    D|          3.0|    (3,[],[])|[0.0, 0.0, 0.0]|
+-----+-----+-------------+-------------+---------------+

## Read the data

In [13]:
interaction_level_df = spark.read.parquet(
                                            # add code here 
                                            "s3://sparkml/EDA02/interaction_level_df.parquet"
                                        )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Adding user level average features

1. user_avg_rating
2. user_n_ratings
3. user_avg_years_betwn_review_and_submission
4. user_avg_prep_time_recipes_reviewed
5. user_avg_n_steps_recipes_reviewed
6. user_avg_n_ingredients_recipes_reviewed

In [14]:
# creating a window partition of dataframe
# for - 1. user_avg_rating
userAvgRatingAgg  = Window.partitionBy("user_id")

# calculate average() of ratings per user
interaction_level_df = interaction_level_df.withColumn("user_avg_rating",
               avg(col("rating")).over(userAvgRatingAgg))

# for - 2. user_n_ratings
interaction_level_df = interaction_level_df.withColumn("user_n_ratings",
               count(col("rating")).over(userAvgRatingAgg))

# for - 3. user_avg_years_betwn_review_and_submission
interaction_level_df = interaction_level_df.withColumn("user_avg_years_betwn_review_and_submission",
               avg(col("years_since_submission_on_review_date")).over(userAvgRatingAgg))

# for - 4. user_avg_prep_time_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_prep_time_recipes_reviewed",
               avg(col("minutes")).over(userAvgRatingAgg))

# for 5. user_avg_n_steps_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_n_steps_recipes_reviewed",
               avg(col("n_steps")).over(userAvgRatingAgg))

# for 6. user_avg_n_ingredients_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_n_ingredients_recipes_reviewed",
               avg(col("n_ingredients")).over(userAvgRatingAgg))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Add columns 

7. user_avg_calories

8. user_avg_total_fat_per_100_cal_recipes_reviewed

9. user_avg_sugar_per_100_cal_recipes_reviewed

10. user_avg_sodium_per_100_cal_recipes_reviewed

11. user_avg_protein_per_100_cal_recipes_reviewed

12. user_avg_saturated_fat_per_100_cal_recipes_reviewed

13. user_avg_carbohydrates_per_100_cal_recipes_reviewed

In [15]:
# for - 7. user_avg_calories
interaction_level_df = interaction_level_df.withColumn("user_avg_calories",
               avg(col("calories")).over(userAvgRatingAgg))


#for 8. user_avg_total_fat_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_total_fat_per_100_cal_recipes_reviewed",
               avg(col("total_fat_per_100_cal")).over(userAvgRatingAgg))
#round(interaction_level_df.filter('user_id == 233044').select('user_avg_total_fat_per_100_cal_recipes_reviewed').first()[0])

#for - 9. user_avg_sugar_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_sugar_per_100_cal_recipes_reviewed",
               avg(col("sugar_per_100_cal")).over(userAvgRatingAgg))

#for 10. user_avg_sodium_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_sodium_per_100_cal_recipes_reviewed",
               avg(col("sodium_per_100_cal")).over(userAvgRatingAgg))

#for 11. user_avg_protein_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_protein_per_100_cal_recipes_reviewed",
               avg(col("protein_per_100_cal")).over(userAvgRatingAgg))

#for 12. user_avg_saturated_fat_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_saturated_fat_per_100_cal_recipes_reviewed",
               avg(col("saturated_fat_per_100_cal")).over(userAvgRatingAgg))

#for 13. user_avg_carbohydrates_per_100_cal_recipes_reviewed
interaction_level_df = interaction_level_df.withColumn("user_avg_carbohydrates_per_100_cal_recipes_reviewed",
               avg(col("carbohydrates_per_100_cal")).over(userAvgRatingAgg))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code. 

assert(round(interaction_level_df.filter('user_id == 601529').select('user_avg_rating').first()[0], 2) == 4.22)
assert(interaction_level_df.filter('user_id == 601529').select('user_n_ratings').first()[0] == 27)
assert(round(interaction_level_df.filter('user_id == 601529').select('user_avg_years_betwn_review_and_submission').first()[0], 2) == 3.51)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_prep_time_recipes_reviewed').first()[0] == 50.3)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_n_steps_recipes_reviewed').first()[0] == 8.8)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_n_ingredients_recipes_reviewed').first()[0] == 8.2)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_total_fat_per_100_cal_recipes_reviewed').first()[0]) == 6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**More Features:**

high_ratings = 5 rating

- `user_avg_years_betwn_review_and_submission_high_ratings`
- `user_avg_prep_time_recipes_reviewed_high_ratings`
- `user_avg_n_steps_recipes_reviewed_high_ratings`
- `user_avg_n_ingredients_recipes_reviewed_high_ratings`

In [17]:
#from pyspark.sql.functions import when
# Create indicator columns for highest ratings 
interaction_level_df = interaction_level_df.withColumn("ind_5_rating",
               when(col("rating") ==5, "1").otherwise("0"))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#user_n_5_ratings
interaction_level_df = (
                        interaction_level_df
                        .withColumn("user_n_5_ratings",
                                    when(col("rating") ==5, count(col("rating")).over(userAvgRatingAgg)).otherwise(None))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
interaction_level_df.filter('user_id == 601529').select('user_id',  'rating', 'user_n_5_ratings').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+----------------+
|user_id|rating|user_n_5_ratings|
+-------+------+----------------+
| 601529|     4|            null|
| 601529|     5|              27|
| 601529|     5|              27|
| 601529|     3|            null|
| 601529|     5|              27|
+-------+------+----------------+
only showing top 5 rows

In [20]:
# Add columns mentioned above 
# hint explore window functions in pyspark.sql
#user_avg_years_betwn_review_and_submission_5_ratings: double (nullable = true)
interaction_level_df = (
                        interaction_level_df
                        .withColumn("user_avg_years_betwn_review_and_submission_5_ratings",
                        when(col("rating") ==5, avg(col("years_since_submission_on_review_date")).over(userAvgRatingAgg))
                        .otherwise(None))
                       )

# `user_avg_prep_time_recipes_reviewed_high_ratings`
#user_avg_prep_time_recipes_reviewed_5_ratings: double (nullable = true)
interaction_level_df = (
                        interaction_level_df
                        .withColumn("user_avg_prep_time_recipes_reviewed_5_ratings",
                        when(col("rating") ==5, avg(col("minutes")).over(userAvgRatingAgg))
                        .otherwise(None))
                       )

# user_avg_n_steps_recipes_reviewed: double (nullable = true)
interaction_level_df = (
                        interaction_level_df
                        .withColumn("user_avg_n_steps_recipes_reviewed_5_ratings",
                        when(col("rating") ==5, avg(col("n_steps")).over(userAvgRatingAgg))
                        .otherwise(None))
                       )

# user_avg_n_ingredients_recipes_reviewed: double (nullable = true)
interaction_level_df = (
                        interaction_level_df
                        .withColumn("user_avg_n_ingredients_recipes_reviewed_5_ratings",
                        when(col("rating") ==5, avg(col("n_ingredients")).over(userAvgRatingAgg))
                        .otherwise(None))
                       )



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("calories_5_ratings",
                                   when(col("rating") ==5, col("calories")).otherwise(None)
                                  )
                       )

interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_calories_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("calories")).over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )
#fat per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("total_fat_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("total_fat_per_100_cal")).otherwise(None)
                                  )
                       )
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_total_fat_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("total_fat_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

# sugar per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("sugar_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("sugar_per_100_cal")).otherwise(None)
                                  )
                       )
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_sugar_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("sugar_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

# sodium per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("sodium_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("sodium_per_100_cal")).otherwise(None)
                                  )
                       )
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_sodium_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("sodium_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

# protein per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("protein_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("protein_per_100_cal")).otherwise(None)
                                  )
                       )
    
    
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_protein_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("protein_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

# saturated_fat per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("saturated_fat_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("saturated_fat_per_100_cal")).otherwise(None)
                                  )
                       )
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_saturated_fat_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("saturated_fat_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

# carbohydrates per 100 calories columns
interaction_level_df = (
                        interaction_level_df.
                        withColumn("carbohydrates_per_100_cal_5_ratings",
                                   when(col("rating") ==5, col("carbohydrates_per_100_cal")).otherwise(None)
                                  )
                       )
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_carbohydrates_per_100_cal_recipes_reviewed_5_ratings",
                                   when(col("rating") ==5, avg(col("carbohydrates_per_100_cal"))
                                        .over(userAvgRatingAgg)).otherwise(None)
                                  )
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
interaction_level_df.filter('user_id == 601529').select('user_id', 'calories', 'rating', 'calories_5_ratings').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+------+------------------+
|user_id|calories|rating|calories_5_ratings|
+-------+--------+------+------------------+
| 601529|   302.1|     4|              null|
| 601529|   805.4|     5|             805.4|
| 601529|   317.6|     5|             317.6|
| 601529|   629.2|     3|              null|
| 601529|    27.7|     5|              27.7|
+-------+--------+------+------------------+
only showing top 5 rows

In [22]:
# it is possible that you might have added null values in the dataframe as a result of this exercise. 
# Identify the null values and deal with them 
#from pyspark.sql.functions import col,isnan, when, count
NewColumns = ["user_n_5_ratings", 
              "user_avg_years_betwn_review_and_submission_5_ratings",
              "user_avg_prep_time_recipes_reviewed_5_ratings",
              "user_avg_n_steps_recipes_reviewed_5_ratings",
              "user_avg_n_ingredients_recipes_reviewed_5_ratings",
              "calories_5_ratings",
              "user_avg_calories_recipes_reviewed_5_ratings",
              "total_fat_per_100_cal_5_ratings",
              "user_avg_total_fat_per_100_cal_recipes_reviewed_5_ratings",
              "sugar_per_100_cal_5_ratings",
              "user_avg_sugar_per_100_cal_recipes_reviewed_5_ratings",
              "sodium_per_100_cal_5_ratings",
              "user_avg_sodium_per_100_cal_recipes_reviewed_5_ratings",
              "protein_per_100_cal_5_ratings",
              "user_avg_protein_per_100_cal_recipes_reviewed_5_ratings",
              "saturated_fat_per_100_cal_5_ratings",
              "user_avg_saturated_fat_per_100_cal_recipes_reviewed_5_ratings",
              "carbohydrates_per_100_cal_5_ratings",
              "user_avg_carbohydrates_per_100_cal_recipes_reviewed_5_ratings"
             ]
interaction_level_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in NewColumns]
   ).count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [23]:
#user_avg_5_ratings
interaction_level_df = (
                        interaction_level_df.
                        withColumn("user_avg_5_ratings",
                                   when(col("rating") ==5, avg(col("rating"))
                                        .over(userAvgRatingAgg))
                                  )
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Check - All rows with ratings should have non-null values in corresponding user_avg_5_ratings columns
#interaction_level_df.filter(col("user_avg_5_ratings").isNull()).select('rating','user_avg_5_ratings').show(5)
interaction_level_df.filter(col("user_avg_5_ratings").isNull()).count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

313917

In [25]:
interaction_level_df.filter('user_id == 233044').select('user_id', 'rating', 'user_avg_5_ratings').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------------+
|user_id|rating|user_avg_5_ratings|
+-------+------+------------------+
| 233044|     4|              null|
| 233044|     5|               4.7|
| 233044|     4|              null|
| 233044|     5|               4.7|
| 233044|     5|               4.7|
| 233044|     5|               4.7|
| 233044|     5|               4.7|
| 233044|     4|              null|
| 233044|     5|               4.7|
| 233044|     5|               4.7|
+-------+------+------------------+

In [31]:
interaction_level_df.filter('user_id == 233044').select('user_n_5_ratings').first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [47]:
interaction_level_df.groupby("user_n_5_ratings").count().show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------+
|user_n_5_ratings| count|
+----------------+------+
|              65|  1638|
|             188|   426|
|              94|   724|
|             343|   279|
|              25|  3605|
|             124|   910|
|             243|   831|
|             385|   332|
|             149|  1183|
|             541|   930|
|              77|  1133|
|             181|   713|
|             958|   618|
|            1403|   728|
|            1405|  1064|
|             845|   710|
|             902|   801|
|             463|   371|
|            2643|  1804|
|             274|   341|
|             474|   410|
|            1174|   765|
|            4623|  3713|
|            1224|  1011|
|            1452|   822|
|             595|   445|
|            null|313917|
|              56|  1849|
|             150|   805|
|              79|   858|
+----------------+------+
only showing top 30 rows

In [54]:
# taking a backup 
interaction_level_df_backup = interaction_level_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
interaction_level_df = interaction_level_df.filter(interaction_level_df.user_n_5_ratings.isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
interaction_level_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

812692

In [57]:
# Check values for a given user id

assert(interaction_level_df.filter('user_id == 233044').select('user_n_5_ratings').first()[0] == 7 )
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_years_betwn_review_and_submission_5_ratings').first()[0], 2) == 2.24 )
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_prep_time_recipes_reviewed_5_ratings').first()[0]) == 46)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_n_steps_recipes_reviewed_5_ratings').first()[0], 2) == 7.29)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_n_ingredients_recipes_reviewed_5_ratings').first()[0], 2) == 6.86)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

Traceback (most recent call last):
AssertionError



In [52]:
interaction_level_df.filter('user_id == 233044').select('user_n_5_ratings').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|user_n_5_ratings|
+----------------+
|              10|
|            null|
|              10|
|              10|
|            null|
|              10|
|            null|
|              10|
|              10|
|              10|
+----------------+

In [38]:
interaction_level_df.filter('user_id == 233044').select('user_avg_years_betwn_review_and_submission_5_ratings').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------+
|user_avg_years_betwn_review_and_submission_5_ratings|
+----------------------------------------------------+
|                                                null|
|                                  1.7368279569999998|
|                                  1.7368279569999998|
|                                                null|
|                                  1.7368279569999998|
|                                  1.7368279569999998|
|                                                null|
|                                  1.7368279569999998|
|                                  1.7368279569999998|
|                                  1.7368279569999998|
+----------------------------------------------------+

In [58]:
# years_since_submission_on_review_date_5_ratings
interaction_level_df = (
                        interaction_level_df.
                        withColumn("years_since_submission_on_review_date_5_ratings",
                                   when(col("rating") ==5, col("years_since_submission_on_review_date")).otherwise(None)
                                  )
                       )
#minutes_5_ratings
interaction_level_df = (
                        interaction_level_df.
                        withColumn("minutes_5_ratings",
                                   when(col("rating") ==5, col("minutes")).otherwise(None)
                                  )
                       )

#n_steps_5_ratings
interaction_level_df = (
                        interaction_level_df.
                        withColumn("n_steps_5_ratings",
                                   when(col("rating") ==5, col("n_steps")).otherwise(None)
                                  )
                       )

#n_ingredients_5_ratings
interaction_level_df = (
                        interaction_level_df.
                        withColumn("n_ingredients_5_ratings",
                                   when(col("rating") ==5, col("n_ingredients")).otherwise(None)
                                  )
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturat

## Tags level EDA

In [81]:
interaction_tag_level_df = interaction_level_df.withColumn('individual_tag',F.explode('tags'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
interaction_tag_level_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15835883

In [83]:
tags_ratings_summary = (interaction_tag_level_df
                        .groupBy('individual_tag').agg(F.avg('rating').alias('avg_user_rating'),
#                                                      F.max('rating').alias('max_user_rating'),
#                                                      F.min('rating').alias('min_user_rating'),
                                                       F.count('rating').alias('n_user_ratings'),
                                                       F.countDistinct('id').alias('n_recipes')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
interactions, recipes  =  interaction_level_df.count(), interaction_level_df.agg(F.countDistinct('id')).first()[0]

tags_ratings_summary = (tags_ratings_summary.withColumn("in_percent_recipies", F.col ("n_recipes")/F.lit(recipes))
                                            .withColumn("in_percent_interactions", F.col ("n_user_ratings")/F.lit(interactions)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1. Top ```n``` most rated tags

In [85]:
tags_ratings_summary.sort(tags_ratings_summary.n_user_ratings.desc()).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------+---------+-------------------+-----------------------+
|    individual_tag|avg_user_rating|n_user_ratings|n_recipes|in_percent_recipies|in_percent_interactions|
+------------------+---------------+--------------+---------+-------------------+-----------------------+
|       preparation|            5.0|        810285|   185097| 0.9954020392359318|     0.9970382383485995|
|      time-to-make|            5.0|        798141|   181097| 0.9738911116847359|     0.9820953079395393|
|            course|            5.0|        773348|   175424| 0.9433832386852521|     0.9515880554995004|
|           dietary|            5.0|        649209|   133067| 0.7155986491137498|      0.798837690047398|
|   main-ingredient|            5.0|        621532|   137143| 0.7375182842884185|     0.7647817377308993|
|              easy|            5.0|        454804|   102007| 0.5485662966787128|     0.5596265251780502|
|          occasion|            5.0|        45

Drop tags present in majority of recipes more than 75% rows 

In [86]:
tags_ratings_summary_1 = tags_ratings_summary.filter('in_percent_interactions <= 0.75')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
tags_ratings_summary_1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

527

In [96]:
Percentlile95th = 527 * 0.95

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
Percentlile95th

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

500.65

In [94]:
tags_ratings_summary_1.sort(tags_ratings_summary_1.in_percent_interactions.desc()).show(29)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------+---------+-------------------+-----------------------+
|    individual_tag|avg_user_rating|n_user_ratings|n_recipes|in_percent_recipies|in_percent_interactions|
+------------------+---------------+--------------+---------+-------------------+-----------------------+
|              easy|            5.0|        454804|   102007| 0.5485662966787128|     0.5596265251780502|
|          occasion|            5.0|        450511|    92937|0.49979026845637586|     0.5543440811525153|
|         equipment|            5.0|        357528|    57317| 0.3082354586129754|     0.4399305025766219|
|           cuisine|            5.0|        346353|    73936|  0.397607984856307|    0.42617990579456916|
|  low-in-something|            5.0|        320970|    69141| 0.3718217604543108|    0.39494667106357634|
|         main-dish|            5.0|        268089|    56543|  0.304073094131819|     0.3298777396602895|
|number-of-servings|            5.0|        24

In [103]:
tags_ratings_summary_1.createOrReplaceTempView("S_tag_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
tags_ratings_summary_temp = spark.sql("""SELECT *
FROM S_tag_sql  
order by S_tag_sql.in_percent_interactions  desc limit 29""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
tags_ratings_summary_temp.show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------+---------+-------------------+-----------------------+
|    individual_tag|avg_user_rating|n_user_ratings|n_recipes|in_percent_recipies|in_percent_interactions|
+------------------+---------------+--------------+---------+-------------------+-----------------------+
|              easy|            5.0|        454804|   102007| 0.5485662966787128|     0.5596265251780502|
|          occasion|            5.0|        450511|    92937|0.49979026845637586|     0.5543440811525153|
|         equipment|            5.0|        357528|    57317| 0.3082354586129754|     0.4399305025766219|
|           cuisine|            5.0|        346353|    73936|  0.397607984856307|    0.42617990579456916|
|  low-in-something|            5.0|        320970|    69141| 0.3718217604543108|    0.39494667106357634|
|         main-dish|            5.0|        268089|    56543|  0.304073094131819|     0.3298777396602895|
|number-of-servings|            5.0|        24

In [65]:
# print the quantiles of in_percent_interactions  
get_quantiles(df = tags_ratings_summary_1, # add code here , 
              col_name = "in_percent_interactions", # add code here , 
              quantiles_list = [0.01,0.25,0.5, 0.75,0.8,0.85,0.9,0.95, 0.99])

# Actual output and expected output is not matching -- pending validation -- Jagadish 9/18/2022

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 0, 0.01: 1.2304784592440925e-06, 0.25: 9.105540598406284e-05, 0.5: 0.0018014204643333515, 0.75: 0.008183912232432458, 0.8: 0.5596265251780502, 0.85: 0.5596265251780502, 0.9: 0.5596265251780502, 0.95: 0.5596265251780502, 0.99: 0.5596265251780502, 1.0: 0}

In [106]:
# keep tags appearing in the top 5 percentile 
top_most_frequent_tags = tags_ratings_summary_temp

top_most_frequent_tags.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

29

In [110]:
# extract the list of top frequent tags
top_frequent_tags_list =top_most_frequent_tags.select('individual_tag').rdd.flatMap(lambda x: x).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
top_frequent_tags_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['easy', 'occasion', 'equipment', 'cuisine', 'low-in-something', 'main-dish', 'number-of-servings', '60-minutes-or-less', 'meat', 'taste-mood', 'north-american', '30-minutes-or-less', 'vegetables', 'oven', '4-hours-or-less', 'low-carb', 'holiday-event', 'dinner-party', 'desserts', '15-minutes-or-less', 'healthy', 'american', 'low-sodium', 'beginner-cook', 'low-cholesterol', 'low-calorie', 'comfort-food', 'kid-friendly', 'inexpensive']

In [123]:
interaction_level_df = add_OHE_columns (
 # add code here 
    interaction_level_df, top_frequent_tags_list
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1266.fit.
: org.apache.spark.SparkException: Input column easy does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:245)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:245)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:242)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apach

#### 2.  Bottom ```n``` least rated tags

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|      individual_tag|avg_user_rating|n_user_ratings|n_recipes| in_percent_recipies|in_percent_interactions|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|lamb-sheep-main-dish|            0.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|     black-bean-soup|            3.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|Throw the ultimat...|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|       chicken-stews|            4.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|       desserts-easy|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
only showing top 5 

The above tags are present in 1 recipe in over two hundred thousand. The features we create based on these tags will not teach the model new information. If these tags were one hot encoded, the entire column would be filled with zeros, and only a few rows will have 1s. One hot encoding of these tags is not a good idea. If you come up with an encoding that captures the rarity of these tags, only then can you add these tags to the analysis.

#### 3. Top ```n``` rated tags 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|      individual_tag|avg_user_rating|n_user_ratings|n_recipes| in_percent_recipies|in_percent_interactions|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|  breakfast-potatoes|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|           pork-loin|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|middle-eastern-ma...|            5.0|             2|        1|4.340164752654011E-6|   1.775238791807983E-6|
|Throw the ultimat...|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|       desserts-easy|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
only showing top 5 

Top rated tags have low number of ratings.

In [48]:
get_quantiles (df = # add code here, 
               col_name = # add code here, 
               quantiles_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 104, 0.01: 114.0, 0.05: 180.0, 0.1: 330.0, 0.15: 522.0, 0.2: 806.0, 0.25: 1143.0, 0.5: 6312.0, 0.75: 27839.0, 0.99: 445959.0, 1.0: 630786}

In [49]:
# Drop the tags with less than 100 ratings 
tags_ratings_summary =

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
get_quantiles(df = # add code here, 
              col_name = # add code here,
              quantiles_list = [0.01,0.25,0.5, 0.75,0.8,0.85,0.9,0.95, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 3.6742424242424243, 0.01: 3.962536023054755, 0.25: 4.344224006718102, 0.5: 4.403587357364895, 0.75: 4.451890903550206, 0.8: 4.461777662606893, 0.85: 4.48284199758207, 0.9: 4.503597122302159, 0.95: 4.536699941280094, 0.99: 4.609756097560975, 1.0: 4.822727272727272}

In [52]:
# keep tags above 95 percentile
top_rated_tags_df = 

top_rated_tags_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26

In [53]:
top_rated_tags_list = # collect the list of top rated tags 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check if any of the current tags have been added earlier

In [54]:
set(top_frequent_tags_list) & set(top_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

set()

In [55]:
all_added_columns_set = set(top_frequent_tags_list).union(set(top_rated_tags_list))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
interaction_level_df = add_OHE_columns ( # add code here )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

added column: has_tag_ragu-recipe-contest
added column: has_tag_simply-potatoes2
added column: has_tag_non-alcoholic
added column: has_tag_a1-sauce
added column: has_tag_labor-day
added column: has_tag_punch
added column: has_tag_lettuces
added column: has_tag_cocktails
added column: has_tag_mashed-potatoes
added column: has_tag_smoothies
added column: has_tag_turkey-burgers
added column: has_tag_avocado
added column: has_tag_beverages
added column: has_tag_mango
added column: has_tag_asparagus
added column: has_tag_memorial-day
added column: has_tag_shakes
added column: has_tag_strawberries
added column: has_tag_omelets-and-frittatas
added column: has_tag_salsas
added column: has_tag_greek
added column: has_tag_salads
added column: has_tag_barbecue
added column: has_tag_australian
added column: has_tag_grilling
added column: has_tag_polynesian

#### 3. Bottom ```n``` rated tags 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.0: 3.6742424242424243, 0.01: 3.962536023054755, 0.02: 4.054406964091403, 0.03: 4.096385542168675, 0.04: 4.126760563380282, 0.05: 4.171974522292993, 0.1: 4.238095238095238, 0.15: 4.303225806451613, 0.2: 4.325726141078838, 0.25: 4.344224006718102, 0.5: 4.403587357364895, 0.75: 4.451890903550206, 0.99: 4.609756097560975, 1.0: 4.822727272727272}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check if any of the current tags have been added earlier

In [61]:
all_added_columns_set & set(bottom_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

set()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

added column: has_tag_pressure-canning
added column: has_tag_honduran
added column: has_tag_unprocessed-freezer
added column: has_tag_birthday
added column: has_tag_jellies
added column: has_tag_water-bath

## Final DataFrame

In [63]:
len(interaction_level_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

148

In [ ]:
interaction_level_df.write.mode('overwrite').parquet(# add code here)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…